In [90]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
import shutil
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import time
from tensorflow.keras import datasets, layers, models, losses
from tensorflow.keras.callbacks import EarlyStopping
import random
from keras.optimizers import SGD
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, BatchNormalization

In [6]:
from google.colab import drive
drive.mount('//drive')

Drive already mounted at //drive; to attempt to forcibly remount, call drive.mount("//drive", force_remount=True).


In [10]:
files_list = sorted(os.listdir("/content/drive/MyDrive/WCEBleedGen (updated)/WCEBleedGen (updated)/bleeding/Images"))
print(len(files_list))

1309


In [11]:
IMG_SIZE=256
s = []
labels = []
for file_name in files_list:
    path=("/content/drive/MyDrive/WCEBleedGen (updated)/WCEBleedGen (updated)/bleeding/Images"+"/"+file_name)
    image=cv2.resize(cv2.imread(path),(IMG_SIZE,IMG_SIZE))
    s.append(np.array(image))
    labels.append(1)

files_list2 = os.listdir("/content/drive/MyDrive/WCEBleedGen (updated)/WCEBleedGen (updated)/non-bleeding/images")
for file_name in files_list2:
    path=("/content/drive/MyDrive/WCEBleedGen (updated)/WCEBleedGen (updated)/non-bleeding/images"+"/"+file_name)
    image=cv2.resize(cv2.imread(path),(IMG_SIZE,IMG_SIZE))
    s.append(np.array(image))
    labels.append(0)
features = np.asarray(s)
labels = np.array(labels)

In [50]:
seed_constant=27
np.random.seed(seed_constant)
random.seed(seed_constant)
tf.random.set_seed(seed_constant)
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size = 0.2, shuffle = True, random_state=seed_constant)

In [13]:
cnn = tf.keras.Sequential(
    [
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation="relu",input_shape=(256, 256, 3)),
    BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Dropout(0.10),

    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation="relu"),
    BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Dropout(0.10),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Dropout(0.10),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dropout(0.50),
    tf.keras.layers.Dense(254,activation='relu'),
    tf.keras.layers.Dropout(0.50),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dropout(0.10),
    tf.keras.layers.Dense(64,activation='relu'),
    tf.keras.layers.Dropout(0.10),
    tf.keras.layers.Dense(1, activation="sigmoid")
]
)

In [14]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 batch_normalization (Batch  (None, 256, 256, 32)      128       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 128, 128, 32)      0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 128, 128, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 64)      18496     
                                                                 
 batch_normalization_1 (Bat  (None, 128, 128, 64)      2

In [16]:
early_stopping_callback = EarlyStopping(monitor = 'val_accuracy', patience = 5, mode = 'max', restore_best_weights = True)
cnn.compile(optimizer="adam", loss='binary_crossentropy', metrics=['accuracy'])

In [17]:
history =cnn.fit(features_train, labels_train, epochs=15, batch_size = 32 , shuffle = True, validation_split = 0.20, callbacks=[early_stopping_callback])

Epoch 1/15
53/53 [==============================] - 417s 8s/step - loss: 1.3761 - accuracy: 0.6287 - val_loss: 0.7115 - val_accuracy: 0.5084
Epoch 2/15
53/53 [==============================] - 389s 7s/step - loss: 0.3516 - accuracy: 0.8591 - val_loss: 0.4439 - val_accuracy: 0.8138
Epoch 3/15
53/53 [==============================] - 400s 8s/step - loss: 0.2195 - accuracy: 0.9260 - val_loss: 0.2609 - val_accuracy: 0.9117
Epoch 4/15
53/53 [==============================] - 392s 7s/step - loss: 0.1328 - accuracy: 0.9528 - val_loss: 0.2381 - val_accuracy: 0.9069
Epoch 5/15
53/53 [==============================] - 408s 8s/step - loss: 0.0610 - accuracy: 0.9815 - val_loss: 0.1399 - val_accuracy: 0.9403
Epoch 6/15
53/53 [==============================] - 403s 8s/step - loss: 0.1280 - accuracy: 0.9642 - val_loss: 0.3300 - val_accuracy: 0.8878
Epoch 7/15
53/53 [==============================] - 397s 7s/step - loss: 0.1117 - accuracy: 0.9642 - val_loss: 0.1615 - val_accuracy: 0.9189
Epoch 8/15
53

In [37]:
im=[]
IMG_SIZE=256
files_list = os.listdir("/content/drive/MyDrive/Test Dataset for Auto-WCEBleedGen Challenge version 2/Test Dataset for Auto-WCEBleedGen Challenge version 2/Test Dataset 2/Images")
for file_name in files_list:
    path=("/content/drive/MyDrive/Test Dataset for Auto-WCEBleedGen Challenge version 2/Test Dataset for Auto-WCEBleedGen Challenge version 2/Test Dataset 2/Images"+"/"+file_name)
    image=cv2.resize(cv2.imread(path),(IMG_SIZE,IMG_SIZE))
    im.append(np.array(image))
x_test=np.asarray(im)

In [38]:
l=[]
for i in files_list:
    l.append(i[0:-4])

In [40]:
y_pred=cnn.predict(x_test)

17/17 [==============================] - 32s 2s/step


In [77]:
Y_pred_classes=[]
for i in y_pred:
    if i>0.5:
        Y_pred_classes.append(0)
    else:
        Y_pred_classes.append(1)
accuracy = cnn.evaluate(features_test, labels_test)
print(accuracy)

17/17 [==============================] - 31s 2s/step - loss: 0.2851 - accuracy: 0.9027
[0.285066157579422, 0.902671754360199]


In [78]:
labels_test_=labels_test[:515]

In [80]:
confusion_matrix(labels_test_, Y_pred_classes)

array([[116, 129],
       [103, 167]])

In [82]:
from sklearn.metrics import recall_score

# Calculating the recall score of classifier
print(f"Recall Score of the classifier is: {recall_score(labels_test_,Y_pred_classes)}")
from sklearn.metrics import f1_score

# Calculating the F1 score of classifier
print(f"F1 Score of the classifier is: {f1_score(labels_test_, Y_pred_classes)}")

Recall Score of the classifier is: 0.6185185185185185
F1 Score of the classifier is: 0.5901060070671377


In [83]:
df = pd.DataFrame(list(zip(l, Y_pred_classes)),
               columns =['Image_name', 'Predicted Class label'])
df

,Image_name,Predicted Class label
0,A0050,0
1,A0053,1
2,A0052,1
3,A0054,0
4,A0051,0
...,...,...
510,A0563,1
511,A0561,1
512,A0560,0
513,A0564,1


In [92]:
df.to_csv('SUBMISSION2.csv',index=False)

In [87]:
im1=[]
files_list1 = os.listdir("/content/drive/MyDrive/Test Dataset for Auto-WCEBleedGen Challenge version 2/Test Dataset for Auto-WCEBleedGen Challenge version 2/Test Dataset 1/Marked Images")
for file_name in files_list1:
    path1=("/content/drive/MyDrive/Test Dataset for Auto-WCEBleedGen Challenge version 2/Test Dataset for Auto-WCEBleedGen Challenge version 2/Test Dataset 1/Marked Images"+"/"+file_name)
    image1=cv2.resize(cv2.imread(path1),(IMG_SIZE,IMG_SIZE))
    im1.append(np.array(image1))
x1_test=np.asarray(im1)

In [88]:
l1=[]
for i in files_list1:
    l1.append(i[0:-4])
y1_pred=cnn.predict(x1_test)

2/2 [==============================] - 3s 864ms/step


In [103]:
y1_pred_classes=[]
for i in y1_pred:
    if i>0.5:
        y1_pred_classes.append(0)
    else:
        y1_pred_classes.append(1)
df1 = pd.DataFrame(list(zip(l1, y1_pred_classes)),
               columns =['Image_name', 'Predicted Class label'])
df1

,Image_name,Predicted Class label
0,A0000,0
1,A0006,1
2,A0002,0
3,A0005,1
4,A0013,1
5,A0001,1
6,A0012,1
7,A0014,1
8,A0004,1
9,A0007,1


In [105]:
_labels_test_=labels_test_[:49]

In [106]:
confusion_matrix(_labels_test_, y1_pred_classes)

array([[ 5, 15],
       [ 7, 22]])

In [107]:
from sklearn.metrics import recall_score

# Calculating the recall score of classifier
print(f"Recall Score of the classifier is: {recall_score(_labels_test_,y1_pred_classes)}")
from sklearn.metrics import f1_score

# Calculating the F1 score of classifier
print(f"F1 Score of the classifier is: {f1_score(_labels_test_, y1_pred_classes)}")

Recall Score of the classifier is: 0.7586206896551724
F1 Score of the classifier is: 0.6666666666666667


In [109]:
df1.to_csv('SUBMISSION1.csv',index=False)